# Running Hugging Face accelerate distributed training on Amazon SageMaker

This is a sample code to run a sample using HF accelerate distributed training framework on Amazon SageMaker, this sample use 2 p4d.24xlarge instances which has 16 A100 in total.

This sample we will show how to configure with **FSx for lustre**, FSx is a high performance storage service, could be mount to training instances, easy to use, suitable for large mount dataset (> hundreds GB), you also could use FSx to store your checkpoint and model files.

We will use HuggingFace offcial example, but porting to SageMaker training job instead of using ```accelerate config``` to launch the training job.

In [ ]:
## Update sagemaker python sdk version
!pip install -U sagemaker

In [ ]:
import sagemaker
import boto3
from sagemaker import get_execution_role

sess = sagemaker.Session()
role = get_execution_role()
sagemaker_default_bucket = sess.default_bucket()

account = sess.boto_session.client("sts").get_caller_identity()["Account"]
region = sess.boto_session.region_name

## Prepare a docker image

In [ ]:
%%writefile Dockerfile
## You should change below region code to the region you used, here sample is use us-west-2
From 763104351884.dkr.ecr.us-west-2.amazonaws.com/pytorch-training:2.0.0-gpu-py310-cu118-ubuntu20.04-sagemaker 

## Install packages needed in this NLP example
RUN pip install evaluate datasets==2.3.2 transformers

ENV LANG=C.UTF-8
ENV PYTHONUNBUFFERED=TRUE
ENV PYTHONDONTWRITEBYTECODE=TRUE

## Make all local GPUs visible
ENV NVIDIA_VISIBLE_DEVICES="all"

## enabel EFA
# ENV FI_PROVIDER="efa"
# ENV NCCL_PROTO=simple
# ENV FI_EFA_USE_DEVICE_RDMA=1

# ENV NCCL_LAUNCH_MODE="PARALLEL"
# ENV NCCL_NET_SHARED_COMMS="0"

### ECR Login (Must run before docker build)

In [ ]:
## You should change below region code to the region you used, here sample is use us-west-2
!aws ecr get-login-password --region us-west-2 | docker login --username AWS --password-stdin 763104351884.dkr.ecr.us-west-2.amazonaws.com

**Build image and push to ECR.**

In [ ]:
## define repo name, should contain *sagemaker* in the name
repo_name = "sagemaker-hf-accelerate-demo"

In [ ]:
%%script env repo_name=$repo_name bash

#!/usr/bin/env bash

# This script shows how to build the Docker image and push it to ECR to be ready for use
# by SageMaker.

# The argument to this script is the image name. This will be used as the image on the local
# machine and combined with the account and region to form the repository name for ECR.
# The name of our algorithm
algorithm_name=${repo_name}

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-west-2}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
aws ecr get-login-password --region ${region}|docker login --username AWS --password-stdin ${fullname}

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

### Generate training entrypoint script



In [ ]:
%%writefile accelerate_config.yaml
compute_environment: LOCAL_MACHINE
distributed_type: MULTI_GPU
downcast_bf16: 'no'
gpu_ids: 0,1,2,3,4,5,6,7
machine_rank: 0
main_process_ip: 0.0.0.0
main_process_port: 7777
main_training_function: main
mixed_precision: fp16
num_machines: 2
num_processes: 16

In [ ]:
%%writefile start-train.py

import os
import json
import socket
import yaml

# import sagemaker_ssh_helper
# sagemaker_ssh_helper.setup_and_start_ssh()

if __name__ == "__main__":

    hosts = json.loads(os.environ['SM_HOSTS'])
    current_host = os.environ['SM_CURRENT_HOST']
    host_rank = int(hosts.index(current_host))
    
    master = json.loads(os.environ['SM_TRAINING_ENV'])['master_hostname']
    master_addr = socket.gethostbyname(master)
    
    ########################
    os.environ['NODE_INDEX'] = str(host_rank)
    os.environ['SM_MASTER'] = str(master)
    os.environ['SM_MASTER_ADDR'] = str(master_addr)
    
    os.environ['NCCL_SOCKET_IFNAME'] = 'eth0'
    os.environ['FI_PROVIDER'] = "efa"
    os.environ['NCCL_PROTO'] = "simple"
    os.environ['FI_EFA_USE_DEVICE_RDMA'] = "1"
#     os.environ['NCCL_LAUNCH_MODE'] = "PARALLEL"
#     os.environ['NCCL_NET_SHARED_COMMS'] = "0"
    #########################

    file_name = './accelerate_config.yaml'
    with open(file_name) as f:
        doc = yaml.safe_load(f)
    doc['machine_rank'] = host_rank
    doc['main_process_ip'] = str(master_addr)
    doc['num_machines'] = 2  # how many intances in this training job
    doc['num_processes'] = 16  # how many GPU cards in total
    with open('./accelerate_config.yaml', 'w') as f:
        yaml.safe_dump(doc, f)
        
    os.system("accelerate launch --config_file=accelerate_config.yaml nlp_example.py")


In [ ]:
## The image uri which is build and pushed above
image_uri = "{}.dkr.ecr.{}.amazonaws.com/{}:latest".format(account, region, repo_name)
image_uri

<!-- ### Modify train.py a little about how to save model

Modify the model save methods in training script, change from 

```
trainer.save_state()
safe_save_model_for_hf_trainer(trainer=trainer, output_dir=training_args.output_dir)
```

to

```
tokenizer.save_pretrained(training_args.output_dir)
trainer.save_model(training_args.output_dir)
``` -->

## Create SageMaker Training Job

## normal s3 file channel

In [ ]:
# save train_dataset to s3
training_input_path = f's3://{sess.default_bucket()}/train'
validation_input_path = f's3://{sess.default_bucket()}/validation'
print("uploaded data to:")
print(f"training dataset to: {training_input_path}")
print(f"valication dataset to: {validation_input_path}")

train = sagemaker.inputs.TrainingInput(
        training_input_path, distribution="FullyReplicated", s3_data_type="S3Prefix"
    )
validata = sagemaker.inputs.TrainingInput(
        validation_input_path, distribution="FullyReplicated", s3_data_type="S3Prefix"
    )
data_channels = {"train": train, "validate": validata}

## use FSx

**Before run below cell, you should alread setup FSx in FSx console.**

In [ ]:
## fsx integrate

from sagemaker.inputs import FileSystemInput

# Specify FSx Lustre file system id.
file_system_id = "fs-xxxxxxxx" # Change to your Fsx FS id

# Specify directory path for input data on the file system. 
# You need to provide normalized and absolute path below.
file_system_directory_path = '/msdc3aaa' # Change to your Fsx Mount name which is given in FSx FS details

# Specify the access mode of the mount of the directory associated with the file system. 
file_system_access_mode = 'rw'

# Specify your file system type.
file_system_type = 'FSxLustre'

fsx_fs = FileSystemInput(file_system_id=file_system_id,
                                    file_system_type=file_system_type,
                                    directory_path=file_system_directory_path,
                                    file_system_access_mode=file_system_access_mode)

fsx_channels = {'fsx': fsx_fs}

### Notice
Before run below code, make sure you have :
- Config VPC endpoint for S3, and add related route to below subnet you used
- Config VPC NAT Gateway (if you need pip install during the training or download from internet
    - Add route(0.0.0.0/0 through NAT GW) to route table which is used by below subnet you used
- **Config security group (MUST if you use p4d/p4de instances)**
    - Add inbound rule, allow all traffic in from the security itself
    - Add outbound rule, allow all traffic out to the security itself

In [ ]:
# if you want to ssh to instance through SSH helper
!pip install sagemaker-ssh-helper

In [ ]:
import time
from sagemaker.estimator import Estimator
# from sagemaker_ssh_helper.wrapper import SSHEstimatorWrapper  # <--NEW--

# environment = {
#               'MODEL_S3_BUCKET': sagemaker_default_bucket # The bucket to store pretrained model and fine-tune model
# }

base_job_name = 'hf-accelerate-demo'         

instance_type = 'ml.p4d.24xlarge'

estimator = Estimator(role=role,
#                       dependencies=[SSHEstimatorWrapper.dependency_dir()],  # <--NEW--
                      entry_point='start-train.py',
                      source_dir='./',
                      base_job_name=base_job_name,
                      instance_count=2,
                      instance_type=instance_type,
                      image_uri=image_uri,
#                       environment=environment,
                      subnets=['subnet-56d99b20'], # Should be same vpc with FSx, best to use same subnet with FSx
                      security_group_ids=['sg-e6c3059f'], # Needed when use FSx
                      keep_alive_period_in_seconds=60*15, # Optional to set, Recommend use when debug and fast to relaunch without provision instances and images download, need submit warm pool instances limit increase first
                      disable_profiler=True,
                      debugger_hook_config=False)
# 
# ssh_wrapper = SSHEstimatorWrapper.create(estimator, connection_wait_time_seconds=90)  # <--NEW--

estimator.fit(inputs=fsx_channels)
#estimator.fit(inputs=data_channels)
# instance_ids = ssh_wrapper.get_instance_ids()  # <--NEW--
# print(f'To connect over SSM run: aws ssm start-session --target {instance_ids[0]}')  # <--NEW--

## Reference

[SageMaker Python SDK](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html#sagemaker.estimator.EstimatorBase)

[SSH helper](https://github.com/aws-samples/sagemaker-ssh-helper)

[SageMaker Examples](https://github.com/aws/amazon-sagemaker-examples)